In [1]:
# Install Dependencies
!pip install gdown scikit-learn pandas

import gdown
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

In [2]:
# Download Preprocessed Data

url = 'https://drive.google.com/uc?id=1nUNgJzVOyKywhGODfwN5TfpSGFJE1OTG'
output = 'modeling_dataset.csv'
gdown.download(url, output, quiet=False)

df = pd.read_csv('modeling_dataset.csv')
print("✅ Loaded:", df.shape)

Downloading...
From: https://drive.google.com/uc?id=1nUNgJzVOyKywhGODfwN5TfpSGFJE1OTG
To: /content/modeling_dataset.csv
100%|██████████| 318k/318k [00:00<00:00, 12.1MB/s]

✅ Loaded: (2176, 10)


In [3]:
# ***Regression Model for avg_chunk_size***
features = [
    'packet_count',
    'total_bytes',
    'avg_packet_size',
    'avg_inter_packet_delay',
    'std_inter_packet_delay'
]

X = df[features].fillna(0)

y_reg = df['avg_chunk_size']
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X, y_reg, test_size=0.2, random_state=42)

regressor = RandomForestRegressor(n_estimators=500, random_state=42)
regressor.fit(X_train_r, y_train_r)
y_pred_r = regressor.predict(X_test_r)

mse = mean_squared_error(y_test_r, y_pred_r)
print(f"avg_chunk_size Regressor MSE: {mse:.2f}")

byte_error = np.sqrt(mse)
print(f"avg_chunk_size Regressor Byte Error: {byte_error:.2f}")

avg_chunk_size Regressor MSE: 8554652684.92
avg_chunk_size Regressor Byte Error: 92491.37


In [4]:
# ***Classification Model for Rebuffering (chunk count unexpectedly drops)

# Define rebuffering as chunk_count < 8
# If predicted fewer than 8 chunks were used, this implies rebuffeirng occured!
#
#
df['rebuffered'] = (df['chunk_count'] < 8).astype(int)
y_clf = df['rebuffered']

X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X, y_clf, test_size=0.2, random_state=42)

classifier = LogisticRegression(max_iter=500)
classifier.fit(X_train_c, y_train_c)
y_pred_c = classifier.predict(X_test_c)

acc = accuracy_score(y_test_c, y_pred_c)
report = classification_report(y_test_c, y_pred_c)

print(f"\nRebuffering Classifier Accuracy: {acc:.2f}")
print("Classification Report:\n", report)


Rebuffering Classifier Accuracy: 0.91
Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.96      0.94       348
           1       0.81      0.69      0.75        88

    accuracy                           0.91       436
   macro avg       0.87      0.83      0.85       436
weighted avg       0.90      0.91      0.90       436



In [5]:
!pip install trustee

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.0/49.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.7/328.7 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 104.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 36.5 MB/s eta 0:00:00
  Created wheel for sphinxemoji: filename=sphinxemoji-0.2.0-py3-none-any.whl size=44408 sha256=978dfe42c34497fc1f5a34f892c531f7e8544d0ea5cffffed7c64259423de394
  Stored in directory: /root/.cache/pip/wheels/d4/b0/a5/92b0e0b5611fc4d37f90711172a680959318c4474df5d2718d
  Created wh

In [6]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from trustee import RegressionTrustee

#Assume df is already loaded
# df = pd.read_csv("your_dataset.csv")

#Define features
features = [
    'packet_count',
    'total_bytes',
    'avg_packet_size',
    'avg_inter_packet_delay',
    'std_inter_packet_delay'
]
X = df[features].fillna(0)

#Regression Target: avg_chunk_size
y = df['avg_chunk_size']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Train Random Forest Regressor
regressor = RandomForestRegressor(n_estimators=500, random_state=42)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

#Evaluate the regressor
mse = mean_squared_error(y_test, y_pred)
print(f" avg_chunk_size Regressor MSE: {mse:.2f}")
print(f" avg_chunk_size Regressor RMSE: {np.sqrt(mse):.2f}")
print(f" avg_chunk_size Regressor R²: {r2_score(y_test, y_pred):.2f}")

#Apply Trustee to explain the regressor
trustee = RegressionTrustee(expert=regressor)
trustee.fit(X_train, y_train, num_iter=50, num_stability_iter=10, samples_size=0.3, verbose=True)
dt, pruned_dt, agreement, reward = trustee.explain()
dt_y_pred = dt.predict(X_test)

#Evaluate surrogate tree
print("\n Trustee Fidelity Report (Tree vs Forest):")
print(f"  MSE: {mean_squared_error(y_pred, dt_y_pred):.2f}")
print(f"  R²:  {r2_score(y_pred, dt_y_pred):.2f}")

print("\n Trustee Real Performance Report (Tree vs Ground Truth):")
print(f"  MSE: {mean_squared_error(y_test, dt_y_pred):.2f}")
print(f"  R²:  {r2_score(y_test, dt_y_pred):.2f}")


 avg_chunk_size Regressor MSE: 8554652684.92
 avg_chunk_size Regressor RMSE: 92491.37
 avg_chunk_size Regressor R²: 0.56
Initializing training dataset using RandomForestRegressor(n_estimators=500, random_state=42) as expert model
Expert model score: 0.9490114744027613
Initializing Trustee outer-loop with 10 iterations
########## Outer-loop Iteration 0/10 ##########
Initializing Trustee inner-loop with 10 iterations
########## Inner-loop Iteration 0/50 ##########
Sampling 365 points from training dataset with (1218, 1218) entries
Student model 0-0 trained with depth 16 and 255 leaves:
Student model score: 0.5869931454668511
Student model 0-0 fidelity: 0.5869931454668511
########## Inner-loop Iteration 1/50 ##########
Sampling 365 points from training dataset with (1328, 1328) entries
Student model 0-1 trained with depth 16 and 252 leaves:
Student model score: 0.6468783077658037
Student model 0-1 fidelity: 0.6468783077658037
########## Inner-loop Iteration 2/50 ##########
Sampling 365 po

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  warnings.warn(


In [9]:
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
#from trustee import RegressionTrustee
from trustee.report.trust import TrustReport
OUTPUT_PATH = "out/"
REPORT_PATH = f"{OUTPUT_PATH}/report/trust_report.obj"
os.makedirs(os.path.dirname(REPORT_PATH), exist_ok=True)

trust_report = TrustReport(
        regressor,
        X=X,
        y=y,
        X_train=X_train,
        y_train=y_train,
        X_test=X_test,
        y_test=y_test,
        max_iter=5,
        num_pruning_iter=0,
        train_size=0.7,
        trustee_num_iter=10,
        trustee_num_stability_iter=5,
        trustee_sample_size=0.3,
        analyze_branches=True,
        analyze_stability=True,
        top_k=10,
        verbose=True,
        feature_names=features,
        is_classify=False,
)

trust_report.save(REPORT_PATH)
print(f"\n Report saved to: {REPORT_PATH}")

Running Trust Report...
Preparing data...
Done!
Progress |----------------------------------------------------------------------------------------------------| 0.9% Complete
Done!
Progress |█---------------------------------------------------------------------------------------------------| 1.8% Complete
Fitting blackbox model...
Done!
Blackbox model score report with training data:
R2 Score: 0.5607297300914976
Using Classification Trustee algorithm to extract DT...
Initializing training dataset using RandomForestRegressor(n_estimators=500, random_state=42) as expert model
Expert model score: 0.9453859354330122
Initializing Trustee outer-loop with 5 iterations
########## Outer-loop Iteration 0/5 ##########
Initializing Trustee inner-loop with 5 iterations
########## Inner-loop Iteration 0/10 ##########
Sampling 365 points from training dataset with (1218, 1218) entries
Student model 0-0 trained with depth 18 and 255 leaves:
Student model score: 0.5308720200562438
Student model 0-0 fide

Done!
Plotting...


Streaming output truncated to the last 5000 lines.


Done!
Done!

 Report saved to: out//report/trust_report.obj


<Figure size 3000x300 with 0 Axes>

...

<Figure size 5000x300 with 0 Axes>

<Figure size 5000x300 with 0 Axes>

<Figure size 4000x300 with 0 Axes>

<Figure size 4000x300 with 0 Axes>

<Figure size 4000x300 with 0 Axes>